In [5]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

import nest_asyncio
nest_asyncio.apply()

urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]

from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

# ✅ Set verbose=False to suppress function logs
agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=False
)

agent = AgentRunner(agent_worker)

# === QUERY 1 ===
response1 = agent.query(
    "Tell me about the evaluation dataset used in LongLoRA, "
    "and then tell me about the evaluation results"
)
print("=" * 50)
print(response1)

# === QUERY 2 ===
response2 = agent.query("Give me a summary of both Self-RAG and LongLoRA")
print("=" * 50)
print(response2)

# === QUERY 3 ===
response3 = agent.query(
    "Tell me about the evaluation dataset used in MetaGPT and compare it against SWE-Bench"
)
print("=" * 50)
print(response3)


Getting tools for paper: metagpt.pdf
Getting tools for paper: longlora.pdf
Getting tools for paper: selfrag.pdf
assistant: The evaluation dataset used in LongLoRA is the PG19 test split. 

Regarding the evaluation results, the models in LongLoRA achieve better perplexity with longer context sizes. Increasing the context window size leads to improved perplexity values. The models are fine-tuned on different context lengths, such as 100k, 65536, and 32768, and achieve promising results on these extremely large settings. However, there is some perplexity degradation observed on small context sizes for the extended models, which is a known limitation of Position Interpolation.
assistant: Here are summaries of Self-RAG and LongLoRA:

1. Self-RAG is a framework that enhances the quality and factuality of large language models by incorporating retrieval and self-reflection mechanisms. It enables language models to generate text informed by retrieved passages when necessary and evaluate the ou